In [ ]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -q mmcv==2.0.0rc4 -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 20.8 MB/s eta 0:00:00


In [ ]:
!pip install -q mmsegmentation==1.2.2
!pip install -q mmengine==0.10.3
!pip install -q ftfy==6.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.11.0+cu113 True
1.2.2


In [ ]:
import os
import numpy as np
import torch
import json
import re
from tqdm import tqdm
from PIL import Image
from skimage import morphology, measure, color, img_as_ubyte, img_as_float
from skimage.color import rgb2gray
from scipy import ndimage
from scipy.ndimage import label
import cv2
import matplotlib.pyplot as plt

from mmseg.apis import init_model, inference_model
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset
from mmengine import Config
from mmengine.config import Config


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
#creazione per fare inference con il test riorganizzato come lo hanno i professori
def reorganize_test_data():
    main_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TEST'
    output_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_TEST_riorg'

    # Percorsi di destinazione per immagini e maschere
    dest_images_folder = os.path.join(output_folder, 'images')
    dest_masks_folder = os.path.join(output_folder, 'masks')

    # Crea le cartelle di destinazione se non esistono
    os.makedirs(dest_images_folder, exist_ok=True)
    os.makedirs(dest_masks_folder, exist_ok=True)

    # Ottieni automaticamente i nomi delle sottocartelle presenti in main_folder
    sottocartelle = [d for d in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, d))]

    # Processa ciascuna sottocartella identificata
    for seq in sottocartelle:
        # Percorsi delle cartelle di origine
        src_images_folder = os.path.join(main_folder, seq, 'images')
        src_masks_folder = os.path.join(main_folder, seq, 'masks')

        # Controlla se le cartelle 'images' e 'masks' esistono per la sequenza
        if not os.path.exists(src_images_folder+'_'+seq) or not os.path.exists(src_masks_folder+'_'+seq):
            print(f"Le cartelle 'images' o 'masks' non esistono per la sequenza {seq}. Skipping...")
            continue

        # Percorsi delle cartelle di destinazione per ogni sequenza
        dest_images_seq_folder = os.path.join(dest_images_folder, seq)
        dest_masks_seq_folder = os.path.join(dest_masks_folder, seq)

        # Crea le cartelle di destinazione per la sequenza
        os.makedirs(dest_images_seq_folder, exist_ok=True)
        os.makedirs(dest_masks_seq_folder, exist_ok=True)

        # Sposta le immagini nella nuova struttura
        for image_name in os.listdir(src_images_folder+'_'+seq):
            src_image_path = os.path.join(src_images_folder+'_'+seq, image_name)
            dest_image_path = os.path.join(dest_images_seq_folder, image_name)
            try:
                image = Image.open(src_image_path)
                image.save(dest_image_path)
            except Exception as e:
                print(f"Errore durante il salvataggio dell'immagine {image_name}: {e}")

        # Sposta le maschere nella nuova struttura
        for mask_name in os.listdir(src_masks_folder+'_'+seq):
            src_mask_path = os.path.join(src_masks_folder+'_'+seq, mask_name)
            dest_mask_path = os.path.join(dest_masks_seq_folder, mask_name)
            try:
                mask = Image.open(src_mask_path)
                mask.save(dest_mask_path)
            except Exception as e:
                print(f"Errore durante il salvataggio della maschera {mask_name}: {e}")

        print(f"Riorganizzata la sequenza {seq}")

    print("Riorganizzazione completata.")

# Esegui la funzione
reorganize_test_data()


In [ ]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################
import os
working_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA'
checkpoint_path = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri'

# Results folder
results_folder  = "/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/results_Cambio_parametri_dice_loss_TEST"

# Data folders
test_img_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_TEST_riorg/images'
gt_mask_folder  = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_TEST_riorg/masks'

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [ ]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

# Classes and palette for VERSE dataset
classes = ('background', 'object')
palettePOLYP = [
    (0, 0, 0),       # background - black
    (255, 255, 255)  # object - white
]

@DATASETS.register_module()
class POLYP(BaseSegDataset):
    METAINFO = dict(classes=classes, palette=palettePOLYP)

    def __init__(self, **kwargs):
        super().__init__(img_suffix='.png',
                         seg_map_suffix='.png',
                         reduce_zero_label=False,
                         **kwargs)

In [ ]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################


config_file = os.path.join(checkpoint_path,'unet-s5-d16_fcn_4xb4-160k_cityscapes-512x1024.py')
cfg = Config.fromfile(config_file)
cfg.device='cuda'

# Visualizzazione del config caricato (unet)
print(f'Config:\n{cfg.pretty_text}')
checkpoint_path=os.path.join(checkpoint_path,'iter_2961.pth')
# Initialize the model from the config and the checkpoint
model = init_model(cfg, checkpoint_path, 'cuda:0')

Config:
crop_size = (
    256,
    256,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        256,
        256,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SPLIT_No_organize_CROP_CON_0+ INPAINTING_Normalized'
dataset_type = 'POLYP'
default_hooks = dict(
    checkpoint=dict(by_epoch=False, interval=329, type='CheckpointHook'),
    logger=dict(interval=50, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook'))
default_scope = 'mmseg'
device = 'cuda'
env_cfg = dict(
    cudnn_benchmark=True,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_met

/usr/local/lib/python3.10/dist-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/usr/local/lib/python3.10/dist-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '


Loads checkpoint by local backend from path: /content/drive/MyDrive/VERSIONE FINALE/RETE E PESI FINALI/iter_2961.pth


In [ ]:


# Ottieni la lista delle sequenze
seq_folders = os.listdir(test_img_folder)

# Crea un dizionario per memorizzare le immagini per ogni sequenza
seq_dict = {}

# Itera attraverso le cartelle delle sequenze
for seq in seq_folders:
    seq_images_folder = os.path.join(test_img_folder, seq)

    # Verifica che la cartella esista e contenga immagini
    if os.path.isdir(seq_images_folder):
        # Ottieni la lista delle immagini in quella sequenza
        img_list = os.listdir(seq_images_folder)

        # Verifica che la lista di immagini non sia vuota
        if img_list:
            for img_name in img_list:
                # Assicurati che l'immagine sia un file e non una sottocartella
                if os.path.isfile(os.path.join(seq_images_folder, img_name)):
                    # Aggiungi la sequenza nel dizionario se non è già presente
                    if seq not in seq_dict:
                        seq_dict[seq] = []

                    # Aggiungi l'immagine alla lista della sequenza
                    seq_dict[seq].append(os.path.join(seq_images_folder, img_name))

# Ordinamento delle immagini per ogni sequenza
for seq in seq_dict:
    seq_dict[seq].sort()

# Ora seq_dict conterrà la lista delle immagini per ciascuna sequenza


In [ ]:
# Funzione per calcolare il Dice coefficient (dal codice 1)
def calculate_2d_dice(pred_image, gt_image):
    """Calculate the 2D Dice coefficient between two images."""
    # Flatten the images to 1D arrays
    pred_flat = pred_image.flatten()
    gt_flat = gt_image.flatten()


    # Handle the case where both masks are empty
    if np.sum(pred_flat) == 0 and np.sum(gt_flat) == 0:
        return 1.0  # Consider it a perfect match (both are empty)

    # Compute the intersection between the prediction and the ground truth
    intersection = np.sum(pred_flat * gt_flat)

    # Calculate the Dice coefficient
    dice = (2. * intersection) / (np.sum(pred_flat) + np.sum(gt_flat) + 1e-7)

    return dice

    # Funzione per estrarre l'ultimo numero dal nome del file
def extract_last_number_from_filename(filename):
    """Estrae l'ultimo numero nel nome del file."""
    # Trova tutti i numeri nel nome del file
    numbers = re.findall(r'\d+', filename)
    # Restituisce l'ultimo numero trovato, se esiste, altrimenti 0
    return int(numbers[-1]) if numbers else 0

def count_connected_objects(volume):
    """Count the number of connected components in a binary volume."""
    labeled_volume = measure.label(volume, connectivity=1)
    num_objects = np.max(labeled_volume)
    return num_objects

def calculate_absolute_error(pred_count, gt_count):
    """Calculate the absolute error in counting connected objects."""
    error = abs(gt_count - pred_count)
    return error


# Funzione per calcolare il Temporal Stability Index (TSI)
def calculate_tsi(mask1, mask2):
    """Calculate the Temporal Stability Index (TSI) between two binary masks."""
    # Converte le maschere in formato binario e tipo float64
    mask1 = np.array(mask1, dtype=np.float64)
    mask2 = np.array(mask2, dtype=np.float64)
    area1 = np.sum(mask1)
    area2 = np.sum(mask2)

    # Evita divisioni per zero
    if area1 + area2 == 0:
        return 0.0

    # Calcola il TSI basato sulla variazione dell'area
    tsi = abs(area1 - area2) / (area1 + area2)
    return tsi

def detect_and_remove_specular_highlights(image):

    image_array = np.array(image)
    gray=rgb2gray(image_array)
    gray=(gray * 255).astype(np.uint8)
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.dilate(mask, kernel, iterations=2)
    corrected_image = cv2.inpaint(image_array, mask, 5, cv2.INPAINT_TELEA)

    return corrected_image, mask

def regional_masking_and_inpainting(img, discontinuity_threshold=1 * 10**(-2)):

        # Converti l'immagine in array numpy
        img_array = np.array(img)

        # Converte in toni di grigio per calcolare i gradienti
        gray_image = color.rgb2gray(img_array)
        J = np.array(gray_image)

        # Definire il kernel derivativo base
        hx = np.array([[-1, 0, 1]])
        hy = np.array([[-1], [0], [1]])

        # Ricavare i gradienti orizzontale e verticale
        Jx = ndimage.correlate(J, hx)
        Jy = ndimage.correlate(J, hy)

        # Identifica la riga centrale
        central_row = Jx[Jx.shape[0] // 2, :]

        # Calcola le differenze tra pixel adiacenti
        diff = np.abs(np.diff(central_row))

        # Trova il primo indice con discontinuità (da sinistra)
        left_bound = np.argmax(diff > discontinuity_threshold)
        # Trova il primo indice con discontinuità (da destra)
        right_bound = len(diff) - np.argmax(diff[::-1] > discontinuity_threshold) - 1

        # Applica la modifica dei pixel sull'immagine a colori
        modified_image = img_array.copy()
        modified_image[:, :left_bound, :] = 16  # Imposta a 16 i pixel a sinistra di left_bound
        modified_image[:, right_bound:, :] = 16  # Imposta a 16 i pixel a destra di right_bound

        # Reset a zero i pixel con tutti i canali pari a 16
        reset_image = modified_image.copy()
        mask = (reset_image[:, :, 0] == 16) & (reset_image[:, :, 1] == 16) & (reset_image[:, :, 2] == 16)
        reset_image[mask] = [0, 0, 0]  # Imposta a [0, 0, 0] i pixel corrispondenti

        # Esegui l'inpainting sull'immagine croppata
        corrected_image, specular_mask = detect_and_remove_specular_highlights(reset_image)


        return corrected_image

def normalizza_immagine(img_path):
    # Read grayscale image
    image = Image.open(img_path)
    image_array=np.array(image)
    # Normalize image
    normalized_image = cv2.normalize(
      image_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    return Image.fromarray(normalized_image)


# Funzione per rimuovere regioni connesse minori di una soglia
def remove_small_regions(mask, min_size):
    # Identifica le regioni connesse
    labeled_mask, num_features = label(mask)
    # Crea una maschera vuota
    filtered_mask = np.zeros_like(mask, dtype=np.uint8)
    # Mantieni solo le regioni connesse maggiori della soglia
    for region_label in range(1, num_features + 1):
        region = (labeled_mask == region_label)
        if np.sum(region) >= min_size:
            filtered_mask[region] = 1
    return filtered_mask



In [ ]:

###############################
####### Inference Loop ########
###############################
new_size=(256,256)
discontinuity_threshold=1 * 10**(-2)
percentuale=0.6
# Loop over the subjects and their images
for subject, img_list in seq_dict.items():
    # Create a folder for each subject in the results folder
    subject_folder = os.path.join(results_folder,subject)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)

    ### NB: if you want to apply preprocessing on the 3D volume you need to reconstruct
    # it here and change the loop to cycle over the slices of the volume instead of
    # cycling over the images

    for img_name in tqdm(img_list, desc=f'Processing {subject}'):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(test_img_folder, img_name)

            ###############################
            ##### 2D Pre-Processing #######
            ###############################


            # Carica l'immagine a colori (RGB)
            image = Image.open(img_path)
            if image is None:
              print(f"Errore nel caricamento dell'immagine: {img_path}")



            dimensioni=np.shape(image)
            # Ridimensiona l'immagine risultante a 256x256
            image1=image.resize(new_size)
            corrected_image=regional_masking_and_inpainting(image1, discontinuity_threshold)

            corrected_image_pil = Image.fromarray(img_as_ubyte(corrected_image))



            # Save img to temporary file
            corrected_image_pil.save('tmp_img.png')

            image_modified=normalizza_immagine('tmp_img.png')

            image_modified.save('tmp_img.png')




            ###############################
            #######    Inference   ########
            ###############################


            result = inference_model(model, 'tmp_img.png')

            # Get data from the result
            pred_label = result.pred_sem_seg.data.squeeze()
            pred_label = pred_label.cpu().numpy().astype(np.uint8)
            height, width = pred_label.shape[:2]  # Ottieni altezza e larghezza della maschera
            min_size = (height * width) * (percentuale / 100)  # Calcola l'area minima

            ###############################
            ###### 2D Post-Processing #####
            ###############################

            # Filtra le regioni connesse
            filtered_mask = remove_small_regions(pred_label, min_size=min_size)



            ###############################
            #######   Save results  #######
            ###############################




            # Save the result
            pred_label_img = Image.fromarray(filtered_mask, mode='L')
            #pred_label_img = Image.fromarray(pred_label, mode='L')
            pred_label_img = pred_label_img.resize((dimensioni[1],dimensioni[0]))
            pred_label_img.save(os.path.join(subject_folder, os.path.basename(img_name).replace(".jpg",".png")))


Processing seq11: 100%|██████████| 228/228 [00:32<00:00,  7.09it/s]


In [ ]:
#AGGIUNGE POLIPI

# Imposta i percorsi
results_folder = results_folder
output_folder = results_folder
os.makedirs(output_folder, exist_ok=True)  # Crea la cartella se non esiste


# Funzione per aggiungere un polipo basandosi sulle maschere adiacenti
def add_polyp_between_adjacent_frames(prev_mask, curr_mask, next_mask):
    if np.max(curr_mask) == 0 and np.max(prev_mask) > 0 and np.max(next_mask) > 0:
        intersection_mask = np.logical_and(prev_mask > 0, next_mask > 0)
        if np.any(intersection_mask):
            curr_mask[intersection_mask] = 1
    return curr_mask

# Funzione per elaborare tutte le maschere di una sequenza
def process_masks_in_sequence(results_folder,seq,sequence_images, output_folder, percentuale=1):
    if len(sequence_images) < 3:
        print(f"Sequenza troppo corta ({len(sequence_images)} immagini) per essere elaborata.")
        return

    for i in tqdm(range(len(sequence_images)), desc="Processing masks", unit="mask"):
        try:
            # Percorso della maschera corrente
            curr_path = sequence_images[i]


            # Ricava solo la base
            base = os.path.basename(curr_path)

             # Carica la maschera corrente
            curr_mask = np.array(Image.open(os.path.join(results_folder,seq,base.replace('.jpg','.png'))))

            # Calcola l'area minima per la rimozione delle piccole regioni
            height, width = curr_mask.shape[:2]
            min_size = (height * width) * (percentuale / 100)

            if i == 0 or i == len(sequence_images) - 1:
                # Prima o ultima immagine: solo rimozione di piccole regioni
                continue
            else:
                # Immagini centrali: rimozione piccole regioni + aggiunta polipi
                base_i_meno_1 = os.path.basename(sequence_images[i - 1])
                prev_mask = np.array(Image.open(os.path.join(results_folder,seq,base_i_meno_1.replace('.jpg','.png'))))
                base_i_piu_1 = os.path.basename(sequence_images[i + 1])
                next_mask = np.array(Image.open(os.path.join(results_folder,seq,base_i_piu_1.replace('.jpg','.png'))))

                processed_mask = add_polyp_between_adjacent_frames(prev_mask, curr_mask, next_mask)

            # Salva la maschera aggiornata
            relative_path = os.path.join(seq, base.replace('.jpg','.png'))
            output_path = os.path.join(output_folder, relative_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            Image.fromarray(processed_mask).save(output_path)
        except Exception as e:
            print(f"Errore durante l'elaborazione dell'immagine: {curr_path}")
            print(f"Dettagli dell'errore: {e}")

# Itera su tutte le sequenze nel dizionario
for seq, sequence_images in seq_dict.items():
    print(f"Processing sequence: {seq}")
    process_masks_in_sequence(results_folder,seq,sequence_images, output_folder, percentuale=2)


Processing sequence: seq6


Processing masks: 100%|██████████| 91/91 [00:04<00:00, 18.91mask/s]


Processing sequence: seq14


Processing masks: 100%|██████████| 249/249 [00:10<00:00, 23.53mask/s]


Processing sequence: seq11


Processing masks: 100%|██████████| 228/228 [00:06<00:00, 36.63mask/s]


In [ ]:
# Lista delle sequenze (sub-cartelle)
sequences = os.listdir(gt_mask_folder)

# Dizionario per salvare i risultati per sequenza
evaluation_results = {}

# Calcola le metriche per tutte le immagini
for seq in sequences:
  # Costruisci i percorsi per le sottocartelle
  gt_seq_folder = os.path.join(gt_mask_folder, seq)
  result_seq_folder = os.path.join(results_folder, seq)

  # Ordinare le liste di maschere manuali e automatiche
  list_mask_manual = sorted(os.listdir(gt_seq_folder), key=extract_last_number_from_filename)
  list_mask_auto = sorted(os.listdir(result_seq_folder), key=extract_last_number_from_filename)

  # Assicurati che la lista delle maschere sia la stessa in entrambe le cartelle
  if len(list_mask_manual) != len(list_mask_auto):
      raise ValueError(f"Il numero di maschere nelle cartelle {gt_seq_folder} e {result_seq_folder} non corrispondono!")

  # Liste per raccogliere le metriche per questa sequenza
  Dices = []
  absolute_errors = []
  TSIs = []  # Lista per raccogliere i valori di TSI

  # Lista per raccogliere le metriche per ogni immagine
  images_metrics = []

  # Ciclo su ogni immagine e calcola le metriche
  prev_auto_mask = None  # Maschera automatica precedente per il calcolo del TSI
  for manual_mask_path, auto_mask_path in tqdm(zip(list_mask_manual, list_mask_auto)):
      # Rimuoviamo "_mask" dal nome del file per fare il confronto
      if "_mask" in manual_mask_path:
          manual_mask_name = manual_mask_path.replace("_mask", "")
      else:
          manual_mask_name = manual_mask_path

      # Verifica che i nomi corrispondano tra ground truth e risultato
      if manual_mask_name.replace(".jpg","") != auto_mask_path.replace(".png",""):
          raise ValueError(f"Il nome della maschera nel ground truth ({manual_mask_path}) non corrisponde con il nome nel risultato ({auto_mask_path})!")

      # Lettura delle maschere
      manual_mask = Image.open(os.path.join(gt_seq_folder, manual_mask_path))
      auto_mask = Image.open(os.path.join(result_seq_folder, auto_mask_path))

      auto_mask = np.array(auto_mask)
      manual_mask = np.array(manual_mask.convert('L'))
      manual_mask = np.where(manual_mask > 127, 1, 0).astype(np.uint8)

      # Calcola il Dice coefficient
      Dice = calculate_2d_dice(manual_mask, auto_mask)

      # Aggiungi il Dice alla lista
      Dices.append(Dice)

      # Conta gli oggetti connessi
      pred_count = count_connected_objects(auto_mask)
      gt_count = count_connected_objects(manual_mask)

      # Calcola l'errore assoluto
      absolute_error = calculate_absolute_error(pred_count, gt_count)
      absolute_errors.append(absolute_error)

      # Calcola il TSI se esiste una maschera precedente
      if prev_auto_mask is not None:
          tsi = calculate_tsi(prev_auto_mask, auto_mask)
          TSIs.append(tsi)
      else:
          tsi = None  # Nessun valore TSI per il primo frame

      # Aggiorna la maschera precedente
      prev_auto_mask = auto_mask

      # Aggiungi le metriche di questa immagine alla lista per questa sequenza
      images_metrics.append({
          'image': manual_mask_name,  # Nome dell'immagine
          'Dice': float(Dice),
          'Absolute Error': float(absolute_error),
          'TSI': float(tsi) if tsi is not None else None
      })

  # Calcola media e deviazione standard per le metriche per la sequenza
  evaluation_results[seq] = {
      'metrics': {
          'Dice': {
              'mean': float(np.mean(Dices)),
              'std': float(np.std(Dices))
          },
          'Absolute Error': {
              'mean': float(np.mean(absolute_errors)),
              'std': float(np.std(absolute_errors))
          },
          'TSI': {
              'mean': float(np.mean(TSIs)) if TSIs else None,
              'std': float(np.std(TSIs)) if TSIs else None
          }
      },
      'images': images_metrics  # Aggiungi le metriche per ogni immagine
  }

# Stampa i risultati
for seq, metrics in evaluation_results.items():
  print(f"\nRisultati per la sequenza {seq}:")
  for metric, values in metrics['metrics'].items():
      print(f"  {metric} - Media: {values['mean']:.4f}, Deviazione standard: {values['std']:.4f}")

# Salva i risultati dell'evaluazione su un file JSON
evaluation_json_path = os.path.join(results_folder, 'performance_dice_per_paziente.json')
with open(evaluation_json_path, 'w') as json_file:
  json.dump(evaluation_results, json_file, indent=4)

91it [00:04, 18.68it/s]
249it [00:12, 20.27it/s]
228it [00:03, 58.38it/s]


Risultati per la sequenza seq6:
  Dice - Media: 0.7191, Deviazione standard: 0.4026
  Absolute Error - Media: 0.3516, Deviazione standard: 0.6525
  TSI - Media: 0.0937, Deviazione standard: 0.2285

Risultati per la sequenza seq14:
  Dice - Media: 0.6141, Deviazione standard: 0.3973
  Absolute Error - Media: 2.5663, Deviazione standard: 6.2515
  TSI - Media: 0.3001, Deviazione standard: 0.3660

Risultati per la sequenza seq11:
  Dice - Media: 0.5183, Deviazione standard: 0.4537
  Absolute Error - Media: 0.4079, Deviazione standard: 0.5003
  TSI - Media: 0.0401, Deviazione standard: 0.1261


In [ ]:
# Leggi il file JSON
with open(os.path.join(results_folder,'performance_dice_per_paziente.json'), 'r') as file:
    data = json.load(file)

# Inizializza dizionari per salvare i valori di tutte le metriche globalmente
metrics = {"Dice": [], "Absolute Error": [], "TSI": []}

# Itera su tutte le sequenze nel file JSON
for seq_name, seq_data in data.items():
    # Estrarre le immagini e le loro metriche
    images = seq_data['images']

    # Popola i dizionari con i valori per ogni immagine
    for img in images:
        for metric in metrics:
            value = img.get(metric)
            if value is not None:
                metrics[metric].append(value)

# Calcola la media e la deviazione standard globali per ciascuna metrica
overall_stats = {}
for metric, values in metrics.items():
    if values:  # Verifica che ci siano valori per la metrica
        mean = np.mean(values)
        std = np.std(values)
        overall_stats[metric] = {"mean": mean, "std": std}

# Stampa i risultati globali
print("Statistiche globali per: ")
for metric, stats in overall_stats.items():
    print(f"{metric}: Mean = {stats['mean']:.4f}, Std = {stats['std']:.4f}")


Statistiche globali per: 
Dice: Mean = 0.5925, Std = 0.4275
Absolute Error: Mean = 1.3451, Std = 4.2971
TSI: Mean = 0.1628, Std = 0.2977
